In [ ]:
%%bash
mkdir ~/.kaggle
mv kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
cat ~/.kaggle/kaggle.json #credenciales de kaggle generadas con la API

In [ ]:
p_base= !kaggle datasets download -d jerzydziewierz/bee-vs-wasp #bajamos de kaggle los datasets de imagenes

In [ ]:
!unzip bee-vs-wasp.zip 

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

In [ ]:
labels = pd.read_csv("kaggle_bee_vs_wasp/labels.csv", dtype=str)
labels

In [ ]:
duplicados = labels[labels.duplicated()]
print(duplicados) #Chequeamos no tener valores duplicados. No tenemos

In [ ]:
import os
labels.path = labels["path"].str.replace("\\","/")
labels.describe()
#reemplazamos la doble barra en el nombre de archivos del dataset de etiquetas porque genera problemas para que flow from dataframe detecte el path

In [ ]:
labels.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.set(style='white',context='notebook')
sns.countplot(labels['label'],palette='viridis');

plt.title('clases'.title(),fontsize=15);
plt.ylabel('count'.title(), fontsize=8)
plt.xlabel('seccion'.title(), fontsize=8)

In [ ]:
labels['label'].value_counts(1)

In [ ]:
from sklearn.model_selection import train_test_split

X=labels.drop(columns=['label','id'])
y=labels["label"]

train, val = train_test_split(labels, test_size=.2, random_state=42)
val, test = train_test_split(val, test_size=.5, random_state=42)
print('train:', train.shape, '\nval:', val.shape, '\ntest',  test.shape)

## Probamos un modelo secuencial con data augmentation, regularización y dropout

In [ ]:
#chequeamos valores que tenemos en train
labels["label"].value_counts()

In [ ]:
#generador de imagenes nuevas
directory_path=None
batch_size=64
generador_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = generador_train.flow_from_dataframe(
            train,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            subset="training",
            batch_size=64,
            seed=42,
            clases=None,
            shuffle=True,
            class_mode="categorical",
            target_size=(150, 150),
            color_mode="rgb")

validation_generator = val_datagen.flow_from_dataframe(
            val,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            batch_size=64,
            seed=42,
            classes=None,
            shuffle=False,
            class_mode="categorical",
            target_size=(150, 150),
            color_mode="rgb") 

test_datagen=ImageDataGenerator(rescale=1./255.)
test_g = test_datagen.flow_from_dataframe(test,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col='label',
            has_ext=False,
            target_size=(150, 150),
            class_mode="categorical",
            classes=None,
            batch_size=64,
            color_mode="rgb",
            seed=42,)

In [ ]:
classes = list(train_generator.class_indices.keys())
classes
w = 10
h = 10
fig = plt.figure(figsize=(10, 15))
columns = 4
rows = 8
ax = []
x,y = train_generator.next()
for i in range(columns*rows):
    image = x[i]
    label = classes[list(y[i]).index(1)]
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[-1].set_title(label)
    plt.imshow(image)
plt.show()

In [ ]:
classes

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from google.colab import files
from keras.models import load_model
checkpoint_model3 = ModelCheckpoint('from_scratch/weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_accuracy', verbose=0,
                             save_best_only=False, save_weights_only=False, mode='auto')

#files.download('model3.h5')

reduce_lr=ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=1)
early_stopping=EarlyStopping(monitor='val_loss',min_delta=0.001,patience=4,restore_best_weights=True,verbose=1)

callbacks=[reduce_lr, early_stopping, checkpoint_model3]


In [ ]:
#Probamos una red con data augmentation + regularización y dropout en la última capa
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dropout_rate=0.3
model3 = models.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(512, activation='relu',kernel_regularizer=regularizers.l1(0.001)))
model3.add(Dropout(dropout_rate))
model3.add(layers.Dense(4, activation='softmax'))


In [ ]:
model3.summary()
model3.save('model3.h5')

In [ ]:
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
#Antes de fitear vamos a equilibrar las clases 
from sklearn import preprocessing
lb_classes = preprocessing.LabelBinarizer()

# Ajustamos a la variable labels
lb_classes.fit(labels['label'])
lb_classes.classes_

In [ ]:
# Encodeamos las labels
labels_ohe = pd.DataFrame(lb_classes.transform(labels['label']), columns=lb_classes.classes_, index=labels['id'])
labels_ohe.head()

In [ ]:
class_weight = {k:v for k,v in zip(range(len(lb_classes.classes_)),
                                   (1/labels_ohe.iloc[train.index].sum()) * 100)}

class_weight

In [ ]:
# Visualizamos los pesos definidos
((1/labels_ohe.iloc[train.index].sum()) * 100).plot(kind='bar');

In [ ]:
history3 = model3.fit(
      train_generator,
      steps_per_epoch=train_generator.n//train_generator.batch_size, #train / batch size. 
      epochs=85,
      validation_data=validation_generator,
      validation_steps=validation_generator.n//validation_generator.batch_size,
      class_weight=class_weight,
      callbacks=callbacks)


In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model3, show_shapes=True)

In [ ]:
files.download('model3.h5')

In [ ]:
from keras.models import load_model
model3.save('model3.h5')
#model3 = load_model('model3.h5')

In [ ]:
acc3=history3.history['accuracy']
val_loss3=history3.history['val_loss']
loss3=history3.history['loss']
val_acc3=history3.history['val_accuracy']
epochs3 = range(1, len(acc3) + 1)

plt.figure(figsize=(25,8))
plt.title('Modelo de base')
plt.plot(epochs3,loss3)
plt.plot(epochs3,val_loss3)
plt.xticks(ticks=epochs3)
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training loss','Validation Loss'])

plt.figure(figsize=(25,8))
plt.plot(epochs3,acc3)
plt.plot(epochs3,val_acc3)
plt.xticks(ticks=epochs3)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy','Validation Accuracy']);

In [ ]:
model3.evaluate(test_g)

##Probamos con VGG19

In [ ]:
from keras.applications import VGG19

conv_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))


conv_base.summary()

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

train_datagen_vgg = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True
)

val_datagen_vgg = ImageDataGenerator(preprocessing_function=preprocess_input)

batch_size = 20

       # Train 
train_vgg = train_datagen_vgg.flow_from_dataframe(train,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            subset="training",
            batch_size=batch_size,
            classes=None,
            seed=42,
            shuffle=True,
            class_mode="categorical",
            target_size=(150, 150))


        # Validation generator
val_vgg = val_datagen_vgg.flow_from_dataframe(val,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            batch_size=batch_size,
            seed=42,
            classes=None,
            shuffle=True,
            class_mode="categorical",
            target_size=(150, 150))    

        # Test
test_datagen_vgg=ImageDataGenerator(preprocessing_function=preprocess_input)
test_vgg = test_datagen_vgg.flow_from_dataframe(test,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col='label',
            has_ext=False,
            target_size=(150, 150),
            class_mode="categorical",
            classes=None,
            batch_size=batch_size,
            color_mode="rgb",
            seed=42,
            shuffle=False)

In [ ]:
from tensorflow.keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Add, Dense,Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import he_normal

x_input_vgg = Input(shape=(150, 150, 3))
# Instanciamos la base convolucional de VGG19 pre-entrenada en ImageNet
vgg19_pretrained = VGG19(
    include_top=False,
    weights='imagenet',
    pooling='avg',
    input_shape=((150, 150, 3)),
)

# Freezamos los pesos de la base convolucional
vgg19_pretrained.trainable = False

In [ ]:
# La base convolucional procesa los datos de entrada
d = vgg19_pretrained(x_input_vgg)
d = Dropout(0.5)(d)
# La capa densa procesará la salida de la base convolucional y generará las clasificaciones
d = Dense(4, activation='softmax', name='ResNet_Bee_Wasp')(d)

In [ ]:
# Instanciamos el modelo
vgg19_pretrained = Model(inputs=x_input_vgg, outputs=d, name='VGG19')

In [ ]:
# Observamos el summary
vgg19_pretrained.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(vgg19_pretrained, show_shapes=True)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, restore_best_weights=True, verbose=1)
# Podemos incorporar este callback al listado anterior y trabajar con ambos a la vez
callbacks_list2=[reduce_lr,early_stopping]

In [ ]:
vgg19_pretrained.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
history2 = vgg19_pretrained.fit(train_vgg,
      steps_per_epoch=train_vgg.n//train_vgg.batch_size, #train / batch size. 
      epochs=10,
      validation_data=val_vgg,
      validation_steps=val_vgg.n//val_vgg.batch_size,
      callbacks=callbacks_list2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure(figsize=(12,8))
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

##Usamos ResNet50 con imagenet

In [ ]:
#ResNet50
from keras import layers
from keras import models
from keras import utils
from keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from keras.applications import VGG19
from keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Add, Dense,Flatten
from tensorflow.keras.initializers import he_normal

In [ ]:
x_input = Input(shape=(224, 224, 3)) #definimos el input

In [ ]:
# Instanciamos la base convolucional de ResNet50 pre-entrenada en ImageNet
resnet50_pretrained = ResNet50(
    include_top=False,
    weights='imagenet',
    pooling='avg',
    input_shape=((224, 224, 3)),
)

In [ ]:
for layer in resnet50_pretrained.layers:
    print(layer.name)

In [ ]:
# Freezamos los pesos de la base convolucional
resnet50_pretrained.trainable = False

In [ ]:
# La base convolucional procesa los datos de entrada
x = resnet50_pretrained(x_input)
x = Dropout(0.3)(x)
# La capa densa procesará la salida de la base convolucional y generará las clasificaciones
x = Dense(4, activation='softmax', name='ResNet_Bee_Wasp')(x)

# Instanciamos el modelo
resnet50_pretrained = Model(inputs=x_input, outputs=x, name='ResNet50')

In [ ]:
# Observamos el summary
resnet50_pretrained.summary()

In [ ]:
plot_model(resnet50_pretrained, show_shapes=True)

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

train_datagen_resnet = ImageDataGenerator(
    # Incluimos el preprocesamiento propio de ResNet50 en nuestro generador
    preprocessing_function=preprocess_input,
    horizontal_flip=True
)

val_datagen_resnet = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
# Creamos la instancias de flow_from_dataframe()

train_generator50 = train_datagen_resnet.flow_from_dataframe(train,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            subset="training",
            batch_size=32,
            seed=42,
            clases=None,
            shuffle=True,
            class_mode="categorical",
            target_size=(224, 224),
            color_mode="rgb")

val_generator50 = val_datagen_resnet.flow_from_dataframe(val,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col="label",
            has_ext=False,
            batch_size=32,
            seed=42,
            classes=None,
            shuffle=False,
            class_mode="categorical",
            target_size=(224, 224),
            color_mode="rgb") 

test_datagen_resnet=ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator50 = test_datagen_resnet.flow_from_dataframe(test,
            directory="kaggle_bee_vs_wasp/",
            x_col="path",
            y_col='label',
            has_ext=False,
            target_size=(224, 224),
            class_mode="categorical",
            classes=None,
            batch_size=32,
            color_mode="rgb",
            seed=42,
            shuffle=False)

In [ ]:
resnet50_pretrained.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=['acc'])

In [ ]:
# Instanciamos nuestro objeto early_stopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True, verbose=1)

# Instanciamos nuestro objeto chekpoint
checkpoint = ModelCheckpoint('pretrained/weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0,
                             save_best_only=True, save_weights_only=False, mode='auto')

# Definimos una lista de callbacks
es = [early_stopping]

In [ ]:
# Entrenamos la capa densa de nuestro modelo pre-entrenado
history_resnet = resnet50_pretrained.fit(train_generator50,
                                  steps_per_epoch=train_generator50.n//train_generator50.batch_size,
                                  epochs=30,
                                  validation_data=val_generator50,
                                  validation_steps=val_generator50.n//val_generator50.batch_size,
                                  callbacks=es)

In [ ]:
acc_r= history_resnet.history['acc']
val_acc_r = history_resnet.history['val_acc']
loss_r = history_resnet.history['loss']
val_loss_r = history_resnet.history['val_loss']
epochs_r = range(1, len(acc_r) + 1)
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
plt.plot(epochs_r, acc_r, label='Training acc')
plt.plot(epochs_r, val_acc_r, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure(figsize=(12,8))
plt.plot(epochs_r, loss_r, label='Training loss')
plt.plot(epochs_r, val_loss_r, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
resnet50_pretrained.evaluate(test_generator50)

In [ ]:
# Predecimos
val_pred_resnet = resnet50_pretrained.predict(val_generator50)

# Obtenemos la matriz de etiquetas reales de validación
val_labels_resnet = lb_classes.transform(val['label'])

# Evaluamos las shapes de los arrays resultantes
val_pred_resnet.shape, val_labels_resnet.shape

In [ ]:
#generar best_threshold
threshold = np.arange(0.1, 0.9, 0.05)

# Definimos listas vacíos donde iremos volcando los resultados
acc_ = []
accs = []
best_threshold = np.zeros(val_pred_resnet.shape[1])

# Iteramos sobre las distintas clases
for i in range(len(best_threshold)):
    # Indexamos las probabilidades predichas por clase
    y_prob = np.array(val_pred_resnet[:, i])
    
    # Iteramos sobre los posibles umbrales
    for j in threshold:
        # Binarizamos las predicciones de acuerdo al umbral
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        y_pred = np.array(y_pred)
        # Calculamos accuracy
        acc = accuracy_score(val_labels_resnet[:, i], y_pred)
        acc_.append(acc)
    
    # Volcamos los resultados en las listas generales
    acc_  = np.array(acc_)
    index = np.where(acc_==acc_.max())
    accs.append(acc_.max())
    best_threshold[i] = threshold[index[0][0]]
    # Vaciamos la lista de para volver a iterar
    acc_ = []

In [ ]:
accs

In [ ]:
best_threshold

In [ ]:
y_pred_resnet = np.array([[1 if val_pred_resnet[i, j] >= best_threshold[j] else 0 for j in range(val_labels_resnet.shape[1])] for i in range(len(val_labels_resnet))])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Definimos listas vacías para ir volcando las métricas por clase
accuracy_per_class = []
precision_per_class = []
recall_per_class = []
f1_per_class = []

# Definimos un listado de etiquetas de clase
classes = list(lb_classes.classes_)

for i in range(val_labels_resnet.shape[1]):
    # Computamos las métricas de evaluación por clase y hacemos un append a la lista correspondiente
    accuracy_per_class.append(accuracy_score(val_labels_resnet[:, i], (val_pred_resnet[:, i] > 0.5).astype(int)).round(2))
    precision_per_class.append(precision_score(val_labels_resnet[:,  i], (val_pred_resnet[:, i] > 0.5).astype(int)).round(2))
    recall_per_class.append(recall_score(val_labels_resnet[:,  i], (val_pred_resnet[:, i] > 0.5).astype(int)).round(2))
    f1_per_class.append(f1_score(val_labels_resnet[:,  i], (val_pred_resnet[:, i] > 0.5).astype(int)).round(2))
    
    print(classes[i])
    print('Accuracy:', accuracy_per_class[i], 'Precision:', precision_per_class[i],
          'Recall:', recall_per_class[i], 'F1:', f1_per_class[i])
    
    # Visualizamos la matriz de confusión por clase
    cm  = confusion_matrix(val_labels_resnet[:,  i], (val_pred_resnet[:, i] > 0.5).astype(int))
    sns.heatmap(cm, annot=True, cmap='Purples', fmt='.0f', cbar=False, square=True,)
    plt.ylabel('Etiqueta real')
    plt.xlabel('Etiqueta predicha')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
# Predecimos

test_pred_resnet = resnet50_pretrained.predict(test_generator50)

# Tomamos los umbrales óptimos obtenidos en validación para mapear probabilidades a etiquetas de clase
y_pred_resnet = np.array([[1 if test_pred_resnet[i, j] >= best_threshold[j] else 0 for j in range(test_pred_resnet.shape[1])] for i in range(len(test_pred_resnet))])

# Creamos un DF para facilitar luego la visualización de las predicciones
y_pred_resnet = pd.DataFrame(y_pred_resnet, index=test.index)

# Evaluamos las shapes de los arrays resultantes
y_pred_resnet.shape


In [ ]:
import time
from IPython.display import clear_output
from tensorflow.keras.preprocessing.image import load_img, img_to_array
data_dir= "/content/kaggle_bee_vs_wasp"

n = 50
bichos = list()

for i in range(n):
    random_bichos = np.random.choice(test['path'])
    while random_bichos in bichos:
        random_bichos = np.random.choice(test['path'])
    bichos.append(random_bichos)

    # Imagen original
    random_bichos_path = os.path.join(data_dir, random_bichos)
    test_image = load_img(random_bichos_path, target_size=(224, 224))
    test_image = img_to_array(test_image)
 
    # Predicción
    prediction = y_pred_resnet.loc[test.loc[test['path'] == random_bichos].index[0]]
    predicted_labels = (prediction > 0.5).astype(int)

    # Etiquetas
    labels = lb_classes.classes_[predicted_labels > 0]

    # Visualizamos la imagen y su predicción
    title = "{}\n Etiquetas reales = {}\nEtiquetas predichas = {}" \
                .format(test.loc[test['path'] == random_bichos, 'path'].values[0],
                        test.loc[test['path'] == random_bichos, 'label'].values[0],
                        labels)

    plt.imshow(plt.imread(os.path.join(data_dir, random_bichos)))
    plt.title(title)
    plt.axis('off')
    plt.show();
    
    if i > 0 and i % 6 == 0:
        time.sleep(20)
        clear_output(wait=True)

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

# Path a la imagen
img_path = '/content/kaggle_bee_vs_wasp/bee1/10007154554_026417cfd0_n.jpg'

# `img` tipo PIL de 224x224
img = image.load_img(img_path, target_size=(224, 224))

# `x`, float32 Numpy array de (224, 224, 3)
x = image.img_to_array(img)

# Le agregamos una dimesión con np.expand_dims()
# para tener un "batch" de (1, 224, 224, 3)
x = np.expand_dims(x, axis=0)

# Aplicamos una normalización por canal
x = preprocess_input(x)

In [ ]:
conv5_block3_3_conv

In [ ]:
from tensorflow.keras import layers

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # Primero, creamos un modelo que mapea de una imagen de input a las activaciones
    # de la última capa convolucional
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = models.Model(model.inputs, last_conv_layer.output)

    # Segundo, creamos un modelo que mapea las activaciones de la última capa
    # convolucional a las predicciones de clase final
    classifier_input = layers.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = models.Model(classifier_input, x)

    # Luego, computamos el gradiente de la clase con mayor probabilidad para la imagen
    # de input con respecto a las activaciones de la última capa convolucional
    with tf.GradientTape() as tape:
        # Obtenemos las activaciones de la última capa convolucional y la observamos
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Computamos la predicción de clase
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # Calculamos el gradiente de la clase predicha con respecto
    # a los feature map de salida de la última capa convolucional
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # Vector en que cada elemento es la intensidad media
    # del gradiente sobre un canal de feature map específico
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiplicamos cada canal en el feature map por
    # "qué tan importante cada canal es en relación a la clase predicha"
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # La media channel-wise del feature map resultante
    # es nuestro heatmap de activación de clase
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # A los fines de la visualización, normalizamos el heatmap entre 0 y 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
# Imprimimos la clase predicha
preds = resnet50_pretrained.predict(x)

# Tomamos el feature map de output de la última capa, `block5_conv3` con .get_layer()
last_conv_layer_name = 'ResNet_Bee_Wasp'

# Definimos un listado de las capas del clasificador
classifier_layer_names = [
    'input_4',
    'resnet50',
    'dropout_2',
    'ResNet_Bee_Wasp'
]

# Generamos un heatmap de activación de clase
heatmap = make_gradcam_heatmap(x, resnet50_pretrained, last_conv_layer_name, classifier_layer_names)

# Display
plt.matshow(heatmap);

In [ ]:
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras import backend as K
from heatmap import to_heatmap, synset_to_dfs_ids

from keras.applications.resnet50 import ResNet50, preprocess_input
from heatmap import to_heatmap, synset_to_dfs_ids

def display_heatmap(new_model, img_path, ids, preprocessing=None):
    # The quality is reduced.
    # If you have more than 8GB of RAM, you can try to increase it.
    img = image.load_img(img_path, target_size=(800, 1280))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    if preprocessing is not None:
        x = preprocess_input(x)

    out = new_model.predict(x)

    heatmap = out[0]  # Removing batch axis.

    if K.image_data_format() == 'channels_first':
        heatmap = heatmap[ids]
        if heatmap.ndim == 3:
            heatmap = np.sum(heatmap, axis=0)
    else:
        heatmap = heatmap[:, :, ids]
        if heatmap.ndim == 3:
            heatmap = np.sum(heatmap, axis=2)

    plt.imshow(heatmap, interpolation="none")
    plt.show()


new_model = to_heatmap(resnet50_pretrained)

s = '01776313-n'  # Imagenet code for pulga
ids = synset_to_dfs_ids(s)
display_heatmap(new_model, "/content/kaggle_bee_vs_wasp/bee1/10007154554_026417cfd0_n.jpg", ids, preprocess_input)

In [ ]:
from keras.applications.vgg16 import (preprocess_input, decode_predictions)
from keras.preprocessing import image
from keras.layers.core import Lambda
from keras.models import Sequential
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import sys
import cv2

def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def load_image(path):
    img_path = "/content/kaggle_bee_vs_wasp/bee1/10007154554_026417cfd0_n.jpg"
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)

def compile_saliency_function(model, activation_layer='avg_pool'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])

def modify_backprop(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = VGG16(weights='imagenet')
    return new_model

def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def grad_cam(input_model, image, category_index, layer_name):
    model = Sequential()
    model.add(input_model)

    nb_classes = 1000
    target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
    model.add(Lambda(target_layer,
                     output_shape = target_category_loss_output_shape))

    loss = K.sum(model.layers[-1].output)
    conv_output =  [l for l in model.layers[0].layers if l.name is layer_name][0].output
    grads = normalize(K.gradients(loss, conv_output)[0])
    gradient_function = K.function([model.layers[0].input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis = (0, 1))
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam, (224, 224))
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)

    #Return to BGR [0..255] from the preprocessed image
    image = image[0, :]
    image -= np.min(image)
    image = np.minimum(image, 255)

    cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    cam = np.float32(cam) + np.float32(image)
    cam = 255 * cam / np.max(cam)
    return np.uint8(cam), heatmap

preprocessed_input = load_image(sys.argv[1])


predictions = resnet50_pretrained.predict(preprocessed_input)


predicted_class = np.argmax(predictions)
cam, heatmap = grad_cam(resnet50_pretrained, preprocessed_input, predicted_class, "block5_conv3")
cv2.imwrite("gradcam.jpg", cam)

register_gradient()
guided_model = modify_backprop(resnet50_pretrained, 'GuidedBackProp')
saliency_fn = compile_saliency_function(guided_model)
saliency = saliency_fn([preprocessed_input, 0])
gradcam = saliency[0] * heatmap[..., np.newaxis]
cv2.imwrite("guided_gradcam.jpg", deprocess_image(gradcam))